In [34]:
import os
import glob
import polars as pl

# Specify the directory where your CSV files are located
directory = r'D:\github\localt20\processedData'

batting_data = pl.read_csv(os.path.join(directory, 'batting.csv'))
bowling_data = pl.read_csv(os.path.join(directory, 'bowling.csv'))
fielding_data = pl.read_csv(os.path.join(directory, 'fielding.csv'))
batting_data

player_id,Player,Mat,SR
str,str,i64,f64
"""3c13fc3b""","""E Frimpong""",4,150.0
"""29b89ae8""","""WB Rankin""",50,86.48
"""acdc62f5""","""A Nortje""",42,48.57
"""85d8110a""","""KJ Coetzer""",70,119.21
"""c2ecd4a7""","""N Ahir""",11,55.34
…,…,…,…
"""7f048519""","""DJ Willey""",43,130.63
"""a9ddd37d""","""MNR Cooray""",7,114.86
"""488b7e22""","""Anoop Orsu""",19,117.1


In [35]:
batting_data['player_id'].unique()

player_id
str
"""cad11a02"""
"""7f74b21d"""
"""441b1b20"""
"""abb7c76c"""
"""6b71e6cf"""
…
"""2e89652e"""
"""2d03c3f8"""
"""17fbaab5"""


In [32]:
player_data = batting_data.join(bowling_data, on=['player_id'], how='inner').drop("Player_right")
# player_data = player_data.filter(player_data['player_id']).unique()
player_data['player_id'].unique()

player_id
str
"""addfb70e"""
"""e174dadd"""
"""59792462"""
"""922e3dcc"""
"""881a9bdd"""
…
"""19758a47"""
"""f208fe04"""
"""63772ee4"""


In [30]:
player_data = player_data.join(fielding_data, on='player_id', how='inner').drop("Player_right").unique()
player_data

player_id,Player,Mat,SR,Overs,Wkts,Econ,D/I
str,str,i64,f64,f64,f64,f64,f64
"""f80682f8""","""Ravinderpal Singh""",35,145.23,1.1,2.0,12.0,0.588
"""31080a88""","""Aslam Khan""",9,108.33,0.0,0.0,0.0,0.444
"""5aeab54c""","""Ihsanullah""",1,142.85,0.0,0.0,0.0,0.0
"""a049c39b""","""Tariq Aziz""",4,85.1,16.0,6.0,4.37,0.0
"""321fcf79""","""S Singh""",1,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…
"""615688b6""","""Usman Najeeb""",20,102.98,70.3,34.0,7.67,0.25
"""d1761087""","""RM West""",5,100.0,19.0,3.0,7.47,0.2
"""7c840f73""","""R Pathan""",14,157.42,2.0,0.0,12.0,0.0


In [31]:
player_data.write_csv(r'D:\github\localt20\processedData\playerStats.csv')